In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#data loading
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

In [ ]:
import keras
from sklearn.model_selection import train_test_split
import time
from keras.utils import to_categorical
import warnings
warnings.filterwarnings("ignore")

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPool2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


In [ ]:
train.shape

In [ ]:
#train
X = train.drop("label",axis=1)
X = X.values.reshape(-1,28,28,1)
#You can use (train.shape[0],28,28,1) or (42000,28,28,1). All of them are same.
y = to_categorical(train["label"]) 
# Dataset include 10 different number so shape of y will be (42000,10)
# Each column represents one number. The name of process is one - hot encoding.
X.shape, y.shape

In [ ]:
#test
test= test.values.reshape(-1,28,28,1)

In [ ]:
#To scale
X=X/255
test = test/255

In [ ]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, random_state = 1845)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:

#Augmentation
datagen = ImageDataGenerator(
                            rotation_range=40,
                            zoom_range=0.2,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            horizontal_flip=True,
                            vertical_flip=True
                            )

datagen.fit(X_train)

In [ ]:
#training model

#Model
batch_size = 128
num_classes = 10
epochs = 500
cnn_mdl = Sequential()

cnn_mdl.add(Conv2D(32, (3,3), strides = 1, padding = "same", activation = "relu", input_shape = (28,28,1)))

cnn_mdl.add(BatchNormalization())

cnn_mdl.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

cnn_mdl.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))

cnn_mdl.add(Dropout(0.1))

cnn_mdl.add(BatchNormalization())

cnn_mdl.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

cnn_mdl.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))

cnn_mdl.add(BatchNormalization())

cnn_mdl.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

cnn_mdl.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))

cnn_mdl.add(Dropout(0.2))

cnn_mdl.add(BatchNormalization())

cnn_mdl.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

cnn_mdl.add(Conv2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))

cnn_mdl.add(Dropout(0.2))

cnn_mdl.add(BatchNormalization())

cnn_mdl.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))


cnn_mdl.add(Flatten())
cnn_mdl.add(Dense(128, activation = "relu"))

cnn_mdl.add(Dense(num_classes, activation = "softmax"))
cnn_mdl.compile(optimizer = "rmsprop" , loss = 'binary_crossentropy' , metrics = ['accuracy'])
cnn_mdl.summary()

In [ ]:
datagen_train = datagen.flow(X_train,y_train, batch_size=32)
datagen_Validation = datagen.flow(X_test,y_test)

In [ ]:
cnn_mdl.fit(datagen_train,
                      epochs = 50, 
                      validation_data=datagen_Validation,
                      callbacks=[learning_rate_reduction])

In [ ]:
#Prediction
pred_test = cnn_mdl.predict(test)
pred_test = np.argmax(pred_test, axis = 1)

#Submission
submissions = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
submissions['Label'] = pred_test
submissions.to_csv('submission.csv', index = False)

> ##### I will try to implement what I learned from udemy lessons. 
Thank you [Merve Ayyuce Kiraz](https://www.udemy.com/course/derin-ogrenmeye-giris/) for what I learned. 